In [1]:
# 1 加载必要的库
import torch
import torch.nn as nn # 神经网络库
import torch.nn.functional as F
import torch.optim as optim # 优化器
from torchvision import datasets,transforms # torchvison操作数据库

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [4]:
# 2 定义超参数
BATCH_SIZE = 256 # 每批处理的数据

# 要想像使用服务器的GPU上进行深度学习加速，就需要将模型放到GPU上，在服务器中这个操作是通过
# device = torch.device("cuda:0")
# model = model.to(device)

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 是否用GPU还是CPU训练(非Mac系的电脑可以这样写)
# 使用Mac系的M芯片进行加速 https://juejin.cn/post/7137891506777489416
DEVICE_MACBOOK = torch.device("mps") 
EPOCHS = 100 # 训练数据集的轮次

In [2]:
# 3 构建pipeline, 对图像做处理
pipeline = transforms.Compose([
    transforms.ToTensor(), # 将图片转换成tensor
    transforms.Normalize((0.1307,),(0.3081,)) # 对图片进行正则化,作用，当模型出现过拟合现象时，降低模型复杂度
])

In [5]:
# 4 下载，加载数据
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("../../dataset/mnist_data/", train=True, download=True, transform=pipeline)
test_set = datasets.MNIST("../../dataset/mnist_data/", train=False, download=True, transform=pipeline)

# 加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) # shuffle 打乱数据
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

### 插入代码，显示MNIST中的图片
#### 1. 读取文件
#### 2. 提取图片
#### 3. 转换图片
#### 4. 显示图片

In [10]:
# 1. 读取文件
with open("../../dataset/mnist_data/MNIST/raw/train-images-idx3-ubyte","rb") as f: # python中读取文件
    file = f.read()

In [6]:
# 2. 提取图片
image1 = [int(str(item).encode('ascii'),16) for item in file[16: 16+784]]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 24, 24, 24, 294, 310, 373, 38, 358, 597, 583, 295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 54, 148, 340, 368, 595, 595, 595, 595, 595, 549, 370, 595, 578, 405, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 568, 595, 595, 595, 595, 595, 595, 595, 595, 593, 147, 130, 130, 86, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 537, 595, 595, 595, 595, 595, 408, 386, 583, 577, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 342, 263, 595, 595, 517, 17, 0, 67, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 1, 340, 595, 1

In [11]:
# 3. 转换图片
import cv2
import numpy as np

image1_np = np.array(image1, dtype=np.uint8).reshape(28,28,1)
print(image1_np.shape)

NameError: name 'image1' is not defined

In [8]:
# 4. 显示图片
cv2.imwrite("digit.jpg",image1_np)

True

In [13]:
# 5 构建网络模型（***）
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # 卷积层定义
        self.conv1 = nn.Conv2d(1, 10, 5) # 1: 灰度图片的通道， 10: 输出通道， 5: kernel
        self.conv2 = nn.Conv2d(10, 20, 3) # 10: 输入通道， 20: 输出通道， 3: kernel
        # 全连接层定义
        self.fc1 = nn.Linear(20*10*10, 500) # 20*10*10: 输入通道， 500: 输出通道
        self.fc2 = nn.Linear(500, 10) # 500: 输入通道， 10: 输出通道
        
    
    # 反向传播
    def forward(self, x):
        input_size = x.size(0) # batch_size
        x = self.conv1(x) # 输入: batch*1*28*28 , 输出: batch*10*24*24 ( 28 - 5 + 1 = 24 )
        x = F.relu(x) # 保持shape不变， 输出: batch*10*24*24
        x = F.max_pool2d(x, 2, 2) # 输入: batch*10*24*24 输出: batch*10*12*12
        
        x = self.conv2(x) # 输入: batch*10*12*12 输出: batch*20*10*10 ( 12 - 3 + 1 = 10 )
        x = F.relu(x)
        
        x = x.view(input_size, -1) # 拉平, -1 自动计算维度， 20*10*10= 2000
        
        x = self.fc1(x) # 输入: batch*2000 输出: batch*500
        x = F.relu(x) # 保持shape不变
        
        x = self.fc2(x) # 输入: batch*500 输出: batch * 10
        
        output = F.log_softmax(x, dim=1) # 计算分类后，每个数字的概率值
        
        return output

In [14]:
# 6 定义优化器
model = Net().to(DEVICE_MACBOOK)

optimizer = optim.Adam(model.parameters())

In [15]:
# 7 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    # 模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        # 部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失（损失就是差距，和真实值的差距）
        loss = F.cross_entropy(output, target) # (cross_entropy：适合交叉熵函数，如果是二分类问题，就用sigmod函数)
        # 找到概率真最大的下标
        #pred = output.max(1, keepdim=True) # 或者 pred = output.argmax(dim=1)
        # 反向传播
        loss.backward()
        # 参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch, loss.item()))

In [24]:
# 8 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad(): # 不会计算梯度，也不会进行反向传播
        for data, target in test_loader:
            #部署到device上
            data, target = data.to(device), target.to(device)
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1] # 值， 索引
            # pred = torch.max(output, dim=1)
            # pred = output.argmax(dim=1)
            # 累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()

            images = data.cpu().numpy()
            labels = target.cpu().numpy()
            pred = pred.cpu().numpy()
            # batchsize * 1 * 28 * 28
            for idx in range(images.shape[0]):
                im_data = images[idx]
                im_label = labels[idx]
                im_pred = pred[idx]
                im_data = im_data.transpose(1, 2, 0)
                
                print('label', im_label)
                print('pred', im_pred)
                cv2.imshow('imdata', im_data)
                cv2.waitKey(0)
        
        test_loss /= len(test_loader.dataset)
        print("Test -- Average loss : {:.4f}, Accuracy : {:.3f}\n".format(test_loss, 100.0 * correct / len(test_loader.dataset)))

In [ ]:
# 9 调用 方法7 / 8
for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE_MACBOOK, train_loader, optimizer, epoch)
    test_model(model, DEVICE_MACBOOK, test_loader)

Train Epoch : 1 	 Loss : 0.030335
label 9
pred [9]
label 8
pred [8]
label 2
pred [2]
label 7
pred [7]
label 0
pred [0]
label 8
pred [8]
label 9
pred [9]
label 3
pred [3]
label 6
pred [6]
label 1
pred [1]
label 2
pred [2]
label 5
pred [5]
label 0
pred [0]
label 4
pred [4]
label 6
pred [6]
label 4
pred [4]
label 7
pred [7]
label 6
pred [6]
label 9
pred [9]
label 7
pred [7]
label 1
pred [1]
label 7
pred [7]
label 7
pred [7]
label 2
pred [2]
label 5
pred [5]
label 7
pred [7]
label 4
pred [4]
label 3
pred [3]
label 4
pred [4]
label 6
pred [6]
label 9
pred [2]
label 9
pred [9]
label 3
pred [3]
label 3
pred [3]
label 0
pred [0]
label 0
pred [0]
label 7
pred [7]
label 1
pred [1]
label 6
pred [6]
label 5
pred [5]
label 7
pred [7]
label 3
pred [3]
label 1
pred [1]
label 0
pred [0]
label 1
pred [1]
label 6
pred [6]
label 2
pred [2]
label 6
pred [6]
label 3
pred [3]
label 8
pred [8]
label 8
pred [8]
label 7
pred [7]
label 6
pred [6]
label 4
pred [4]
label 1
pred [1]
label 8
pred [8]
label 3
pred [